In [3]:
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import dask
dask.config.set({'array.slicing.split_large_chunks': False})

In [7]:
# Generate dictionary of all model names and file paths 
import os
import pandas as pd
    
var = 'rx1day'
ensemble = 'r1i1p1f1'

institution_dir = '/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical'

all_models = os.listdir(institution_dir)

var_files = []
model_names = []

for model in all_models:
    all_files = os.listdir(f'{institution_dir}/{model}')
    for file in all_files:
        if var in file and ensemble in file:
            var_files.append(f'{model}/{file}')
            model_names.append(model)

paths = [f'/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/{f}'
             for f in var_files]

dictionary = {model_names[i]: paths[i] for i in range(len(model_names))}

In [8]:
# Read in each file and save to my directy for regridding 
names = []

for name, path in dictionary.items():
    try:
        d = xr.open_mfdataset(path, combine='by_coords', chunks={'time':-1, 'lat':110, 'lon':110}).rx1dayETCCDI
        if len(d['time'])!=1980:
            d = d.sel(time=slice('1850-01', '2014-12'))
        del d['time']
        if hasattr(d, 'height'):
            del d['height']
        time_month = pd.date_range(start='1850-01',end = '2015-01', freq ='M')
        d.coords['time'] = time_month
        d.to_netcdf(f'/g/data/w40/rl5183/CMIP6_ETCCDI_rx1/{name}_rx1')
        print(f'Model {name} saved')
        names.append(name)
    except OSError:
        # No files read, move on to the next
        continue 

Model MPI-ESM1-2-HR saved
Model MRI-ESM2-0 saved
Model INM-CM5-0 saved
Model FGOALS-g3 saved
Model INM-CM4-8 saved
Model GFDL-CM4 saved
Model EC-Earth3 saved
Model GFDL-ESM4 saved
Model KACE-1-0-G saved
Model MPI-ESM1-2-LR saved
Model NESM3 saved
Model NorESM2-LM saved
Model EC-Earth3-Veg saved
Model MIROC6 saved
Model ACCESS-CM2 saved
Model NorESM2-MM saved
Model BCC-CSM2-MR saved
Model ACCESS-ESM1-5 saved
Model KIOST-ESM saved
Model CanESM5 saved
